* Generate thermal images from raw temperature data

In [1]:
import os
import glob
import numpy as np
import cv2

Class to generate a thermal image from raw temperature data as float numbers in a 1D array

In [ ]:
class thermal_img_gen(object):

    def __init__(self):
        pass
    
    def linear_AGC(self,temp_data):

        min_temp = np.min(temp_data)
        max_temp = np.max(temp_data)
        img_data = np.zeros(len(temp_data))
        for i in range(len(temp_data)):
            img_data[i] = int(( ( 255 * ( temp_data[i] - min_temp) ) ) / (max_temp-min_temp))
        img_data = np.asarray(img_data,dtype=np.uint8)

        return img_data

    def apply_image_proc(self,image_data):
        clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8,8))
        image_data_numpy_8u_2d = np.asarray((image_data)*255,dtype=np.uint8)
        
        image_data_numpy_8u_2d = cv2.medianBlur(image_data_numpy_8u_2d, 3)
        alpha = 1.0 # Contrast control
        beta = 5 # Brightness control
        image_data_numpy_8u_2d = cv2.convertScaleAbs(image_data_numpy_8u_2d, alpha=alpha, beta=beta)
        image_data_numpy_8u_2d = clahe.apply(image_data_numpy_8u_2d)
        image_data_numpy_8u_2d = cv2.fastNlMeansDenoising( image_data_numpy_8u_2d, None, 7, 7, 13 )  
        
        # apply binary thresholding
        ret, thresh = cv2.threshold(image_data_numpy_8u_2d, 150, 255, cv2.THRESH_BINARY)
        # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
        contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)                            
        # draw contours on the original image
        image_data_numpy_8u_2d = cv2.applyColorMap(image_data_numpy_8u_2d, cv2.COLORMAP_JET)
        cv2.drawContours(image=image_data_numpy_8u_2d, contours=contours, contourIdx=-1, color=(255, 255, 255), 
                         thickness=2, lineType=cv2.LINE_AA)
        return image_data_numpy_8u_2d

    def create_thermal_image_from_temperature_data(self,temp_data):
        
        temp_data = np.asarray(temp_data)
        img_data = temp_data/np.max(temp_data)
        img_data = np.power(img_data,4)
        img_data = self.linear_AGC(temp_data)
        img_data = img_data.reshape(288,384)
        img_data = self.apply_image_proc(img_data)
        return img_data